In [1]:
import cv2
import numpy as np
import copy

class Visualize:

    def __init__(self,mat):
        self.mat=mat
        self.row=len(mat)
        self.col=len(mat[0])
        self.image=np.zeros((self.row*50,2*(self.col*50)//3, 1), dtype="uint8")
        self.radius=28
        self.drawHexArray()
    
    def show(self,wait=0):
        cv2.imshow('imaging',self.image)
        cv2.waitKey(wait)
        cv2.destroyAllWindows()

    def drawHexagone(self,center):
        vertices = []
        for i in range(6):
            x = int(center[0] + self.radius * np.cos(np.radians(i * 60 + 30)))
            y = int(center[1] + self.radius * np.sin(np.radians(i * 60 + 30)))
            vertices.append((x, y))
        pts = np.array(vertices, np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(self.image, [pts], isClosed=True, color=150, thickness=2)

    def refactorPoint(self,i,j):
        gap=25 if i%2==0 else 50
        center=(gap+50*j,self.radius+i*40)
        return center

    def drawHexArray(self):
        newMat=[]
        for row in self.mat:
            col=row.split('-')
            while '' in col: col.remove('')
            newMat.append(col)
        n,m=len(newMat),len(newMat[0])
        for i in range(n):
            for j in range(m):
                center=self.refactorPoint(i,j)
                self.drawHexagone(center)
                org=center[0]-12,center[1]+7
                cv2.putText(self.image, newMat[i][j], org, cv2.FONT_HERSHEY_SIMPLEX, fontScale=.8, color=200, thickness=2)

    def showPath(self,list1,speed=200):
        copied_image = copy.deepcopy(self.image)
        for i in range(len(list1)-1):
            x1,y1=list1[i][0],list1[i][1]//2
            x2,y2=list1[i+1][0],list1[i+1][1]//2
            start=self.refactorPoint(x1,y1)
            end=self.refactorPoint(x2,y2)
            cv2.line(self.image, start, end, color=250, thickness=4)
            cv2.imshow('imaging',self.image)
            cv2.waitKey(speed)
        self.image=copied_image
        cv2.destroyAllWindows()

In [2]:
mat=['O-X-X-X-X-X-O-',
 '-X-O-W-O-X-O-O',
 'O-O-O-O-O-O-X-',
 '-O-O-O-X-O-O-O',
 'O-O-X-O-X-O-O-',
 '-O-X-X-X-O-O-X']

In [3]:
from hive import Hive
hive=Hive(mat)
wasp=hive.getWaspPosition()
fNebors=hive.freeNeibors(*wasp)
edges=hive.getEdges()
print('free nebors of wasp:',fNebors)

free nebors of wasp: [(1, 3), (1, 7), (2, 6), (2, 4)]


In [4]:
def difference(list1,list2):
    newList=[]
    for item in list1:
        if item not in list2:
            newList.append(item)
    return newList

In [5]:
def barrierPath(hive,barrier,path,endPoint,placedBarriers,oldNebors=[]): # path should contain atleast starting point
    if barrier<0: return False
    
    
    nebors=hive.blockedNeibors(*path[-1])
    nebors=difference(nebors,oldNebors)
    for nebor in nebors:
        path.append(nebor)
        oldNebors.append(nebor)
        if barrierPath(hive,barrier,path,endPoint,placedBarriers,oldNebors):
            return True
        path.pop()

    freeNebors=hive.freeNeibors(*path[-1])
    barrier-=1
    for nebor in freeNebors:
        path.append(nebor)
        oldNebors.append(nebor)
        placedBarriers.append(nebor)
        if barrierPath(hive,barrier,path,endPoint,placedBarriers,oldNebors):
            return True
        path.pop()
        oldNebors.pop()
        placedBarriers.pop()
    return False

In [6]:
barrier=3
path=[(0,4)]
endPoint=(0,2)
oldNebors=[(0,2),(0,4)]
placedBarriers=[]
barrierPath(hive,barrier,path,endPoint,placedBarriers,oldNebors)

False

In [7]:
hive.blockedNeibors(0,2)

[(0, 4), (1, 1)]